In [44]:
import tkinter as tk
import pandas as pd
from tkinter import filedialog

root = tk.Tk()
file=filedialog.askopenfilename()
root.withdraw()

df=pd.read_excel(file, skiprows=5, skipfooter=2, header=0)

In [48]:
#Deleting 3rd row of each participant
def Neat(df):
    
    df.dropna(subset=['کد ملی'], inplace=True)
    df.reset_index(drop=True, inplace=True)

    #Creating a df from even rows
    df1 = df[df.index%2 == 0]
    df1.iloc[0,[8,9,11]]=["Birth_En","Mobile","Name_En"]
    df1.columns=df1.iloc[0]
    df1=df1.drop(df1.index[0])
    df1.reset_index(drop=True, inplace=True)

    #Creating a df from odd rows
    df2 = df[df.index%2 == 1]
    df2.reset_index(drop=True, inplace=True)
    df2.columns.values[[6,8,9,11]]=["Prov","Birth_Fa","Nat_ID","Name_Fa"]

    #Prearing a neat df
    df_concat = pd.concat([df1, df2], axis=1)
    df_concat=df_concat.iloc[:,[26,11,21,23,8,9,24]]

    return df_concat

In [46]:
#Check and report the errors
def check(df_concat):
    
    import re

    Errors=""
    rows=range(df_concat.shape[0])
    columns=range(df_concat.shape[1])

    for r in rows:
        for c in columns:
            if c==0:
                x=re.findall("^[ا-ی ]*$" ,df_concat.iloc[r,c])
                if x==[]:
                    Errors+= df_concat.iloc[r,c] + " in row number " + str(r+1) + " contains non-Persian characters." + "\n"
            elif c==1:
                x=re.findall("^[a-zA-Z ]*$" ,df_concat.iloc[r,c])
                if x==[]:
                    Errors+= df_concat.iloc[r,c] + " in row number " + str(r+1) + " contains non-English characters." + "\n"
            if c==2:
                x=re.findall("^[ا-ی ]*$" ,df_concat.iloc[r,c])
                if x==[]:
                    Errors+= df_concat.iloc[r,c] + " in row number " + str(r+1) + " contains non-Persian characters." + "\n"
            elif c==3:
                x=re.findall("13[0-9]{2}/[0-9]{2}/[0-9]{2}" ,df_concat.iloc[r,c])
                if x==[]:
                    Errors+= df_concat.iloc[r,c] + " in row number " + str(r+1) + " is not a valid date like 1399/01/01." + "\n" 
            elif c==4:
                x=re.findall("([0-9]|1[1-2])/([0-9]|1[0-9]|2[0-9]|3[0-1])/([19,20][0-9]{2})" ,df_concat.iloc[r,c])
                if x==[]:
                    Errors+= df_concat.iloc[r,c] + " in row number " + str(r+1) + " is not a valid date like 1/1/2000." + "\n" 
            elif c==5:
                x=re.findall("[0-9]{11}" ,df_concat.iloc[r,c])
                if x==[]:
                    Errors+= df_concat.iloc[r,c] + " in row number " + str(r+1) + " is not a valid Mobile No. like 09121234567." + "\n" 
            elif c==6:
                x=re.findall("[0-9]{10}" ,df_concat.iloc[r,c])
                if x==[]:
                    Errors+= df_concat.iloc[r,c] + " in row number " + str(r+1) + " is not a valid National ID." + "\n" 

    print(Errors)

In [49]:
Neat(df)

,Name_Fa,Name_En,Prov,Birth_Fa,Birth_En,Mobile,Nat_ID
0,کیانا سادات درهمی,kiana sadat derhami,تهران,1373/01/05,3/25/1994,09128305769,0440377331
1,الهام رباط جزی,elham robatjazi,خراسان شمالی,1367/05/02,7/24/1988,09391612742,0829827757
2,زهرا حیدری,zahra heidari,چهارمحال و بختیاری,1368/11/18,2/7/1990,09383014890,4670057650
3,نگار ساجدی,negar sajedi,کردستان,1372/01/19,4/8/1993,09182471508,3750247651


In [1]:
#Dataframe to class
class Besmela:
    #def __init__(self, Dict):
        for k, v in Dict.items():
            setattr(self, k, v)

df_class=Besmela(df_dict)
#print(df_dict.items())
#dir(df_class)

print(type(df_class.Birth_Fa))

NameError: name 'Dict' is not defined

In [ ]:
import sqlite3 as sq
conn= sq.connect('competition.db')
cur= conn.cursor()
cur.execute(""" CREATE TABLE athletes_info5 (Name_Fa, Name_En, Prov, Birth_Fa, Birth_En, Mobile, Nat_ID)""")
cur.execute("""INSERT INTO athletes_info VALUES (?,?,?,?,?,?,?)""", df_class)
conn.commit()